In [ ]:
# Note there are other methods to do this
from google.colab import drive
drive.mount('/content/gdrive/')

In [ ]:
import os
# DIR = "/Users/xin/Desktop/Spring 2022/CS 498 Machine Perception/498 final project"
DIR = "/content/gdrive/My Drive/CS 498 Machine Perception/Final_project"
if not os.path.exists(DIR):
    os.makedirs(DIR)
os.chdir(DIR)

DATASET_PATH = DIR + "/people-gait-master"

In [ ]:
import glob
import os
import numpy as np
import seaborn as sns
from tqdm.notebook import tqdm
import matplotlib
%matplotlib inline
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, average_precision_score
import random

from PIL import Image
import torch
from torch import nn
from torch.utils import data
import torchvision
from torchvision import models
from torchvision.transforms import ToTensor
import torchvision.transforms as transforms
import torchvision.transforms.functional as TF
import torch.nn.functional as F
import torch.optim as optim
import pandas as pd
from scipy.spatial.transform import Rotation 

DBSCAN denoise
merge data

In [ ]:
SUB = 5

In [ ]:
class Preprocess():
    def __init__(self, room=2, device_dist=8.55, num_ppl=1, route="fixed_route", data_dir=DATASET_PATH, shuffle=True, denoise=False, augment=2):
        
        self.data_path77 = data_dir + "/room" + str(room) + "/" + str(num_ppl) + "/" + route + "/77Ghz_radar/"
        self.data_path60 = data_dir + "/room" + str(room) + "/" + str(num_ppl) + "/" + route + "/60Ghz_radar/"
        self.num_ppl = num_ppl

        self.files77 = os.listdir(self.data_path77)[:SUB]
        self.files60 = os.listdir(self.data_path60)

        self.data_dict = {}
        self.time_threshold = 0.05
        
        self.t = 32

        H = np.array([[0,-1,0,device_dist],
                    [-1,0,0,0],
                    [0,0,1,0],
                    [0,0,0,1]])

        # data and label for training
        self.data = []
        self.label = []

        def padList(ndarr, LENGTH = 128):
            ret = ndarr.copy()
            for i, arr in enumerate(ndarr):
                if len(arr) < LENGTH:
                    l = int(len(arr)//LENGTH)
                    r = len(arr)%LENGTH
                    if r != 0:
                        ret[i] = list(arr)*l + list(arr[:r])
                    ret[i] = list(arr)*l
                idx = np.round(np.linspace(0, len(arr) - 1, LENGTH)).astype(int)
                ret[i] = arr[idx]
            return list(ret)


        for f_num in range(len(self.files77)):
            f = self.files77[f_num]
            # load and merge two dataset
            df77 = pd.read_csv(self.data_path77 + f)
            df77["time"] = (df77["h"]*60+df77["m.1"])*60 + df77["s"]
            df60 = pd.read_csv(self.data_path60 + f)
            df60 = df60.rename(columns={'Unnamed: 6': "Intensity"})
            df60["time"] = (df60["h"]*60+df60["m.1"])*60 + df60["s"]
            
            ### need to transform the dataset of one ####
            coors = np.array(df60[["X", "Y", "Z"]])
            coors = np.concatenate([coors, np.ones([len(coors),1])], axis=1)
            coors = H @ coors.T
            df60["X"] = coors[0]/coors[-1]
            df60["Y"] = coors[1]/coors[-1]
            df60["Z"] = coors[2]/coors[-1]
            self.data_dict[f] = pd.concat([df77,df60]).sort_values(by="time").reset_index(drop=True)
            # self.data_dict[f] = df77.sort_values(by="time").reset_index(drop=True)
            
            # filter some noise: 1) obj clearly out of range. 2) make two device the same start time
            self.data_dict[f] = self.data_dict[f][np.abs(self.data_dict[f]["X"]) <= device_dist]
            self.data_dict[f] = self.data_dict[f][np.abs(self.data_dict[f]["Y"]) <= device_dist]
            self.data_dict[f] = self.data_dict[f][np.abs(self.data_dict[f]["Z"]) <= device_dist]
            start_t = np.max([np.array(df60["time"])[0], np.array(df77["time"])[0]])
            self.data_dict[f] = self.data_dict[f][np.abs(self.data_dict[f]["time"]) >= start_t]
            self.data_dict[f] = self.data_dict[f].reset_index(drop=True)

            # print(len(self.data_dict[f]))
            if augment != 0:
              for aug_t in range(augment):
                aug = self.data_dict[f].copy()
                aug["time"] = np.array(aug["time"]) + np.min(np.array(aug["time"]))
                deg = np.random.randint(80) - 40
                r = Rotation.from_euler('z', deg, degrees=True)
                R = r.as_matrix()
                coors = np.array(aug[["X", "Y", "Z"]])
                coors = R @ coors.T
                aug["X"] = coors[0]
                aug["Y"] = coors[1]
                aug["Z"] = coors[2]
                self.data_dict[f] = pd.concat([self.data_dict[f], aug])
                # print(len(self.data_dict[f]))
                self.data_dict[f] = self.data_dict[f].sort_values(by="time").reset_index(drop=True)
                # print(len(self.data_dict[f]))
            

            # group data into frames if two data point if less than time threshold
            frame = []
            old_time = self.data_dict[f]["time"][0]
            start_time = float(self.data_dict[f]["time"][0])
            i = 0
            for t in list(self.data_dict[f]["time"]):
                if t != old_time and start_time + self.time_threshold <= float(t):
                    i += 1
                    start_time = t
                frame.append(i)
                old_time = t
            self.data_dict[f]["frames"] = frame

            # make data and label
            df = np.array(self.data_dict[f][["X", "Y", "Z", "Doppler", "Intensity", "frames"]])
            cur_data = padList(np.split(df[:,:5], np.unique(df[:, 5], return_index=True)[1][1:]))

            # group the data for every self.t frames
            length = len(cur_data)
            cur_data = cur_data[length%self.t:]
            cur_data = np.reshape(cur_data, (length//self.t, self.t, 128, 5))
            cur_label = np.zeros([len(cur_data), len(self.files77)])
            cur_label[:,f_num] = 1

            self.data += list(cur_data)
            self.label += list(cur_label)

        if shuffle:
          p = np.random.permutation(len(self.data))
          self.data = np.array(self.data)[p]
          self.label = np.array(self.label)[p]


    def __len__(self):
        return len(self.data)

    def __getitem__(self, index):
        return ToTensor()(self.data[index]), torch.LongTensor(self.label[index])


In [ ]:
class GetTrainigTesting():
  def __init__(self, data, label, split="train", ratio=0.8):
    self.data = []
    self.label = []
    if split == "train":
      self.data = data[:int(len(data)*ratio)]
      self.label = label[:int(len(label)*ratio)]
    else:
      self.data = data[int(len(data)*ratio):]
      self.label = label[int(len(label)*ratio):]

  def __len__(self):
      return len(self.data)

  def __getitem__(self, index):
      return ToTensor()(self.data[index]), torch.LongTensor(self.label[index])

In [ ]:
load_data = Preprocess(room=1, device_dist=8.48, num_ppl=1, route="fixed_route", data_dir=DATASET_PATH, augment=2)
dataset = GetTrainigTesting(load_data.data, load_data.label, split="train", ratio=0.80)
dataset_test = GetTrainigTesting(load_data.data, load_data.label, split="test", ratio=0.80)

In [ ]:
print("dataset.data",np.shape(dataset.data))
print("dataset.label", np.shape(dataset.label))
print("dataset_test.data",np.shape(dataset_test.data))
print("dataset_test.label", np.shape(dataset_test.label))

In [ ]:
# points_3d = dataset.data[0][300]
# fig = plt.figure(figsize=(10, 10))
# ax = plt.axes(projection='3d')
# ax.scatter3D(points_3d[:,0], points_3d[:,1], points_3d[:,2], c='b',label='Estimated 3D Points')
# ax.view_init(azim=30, elev=20)
# ax.legend(loc='best')
# ax.set_title("View1")
# plt.show()

In [ ]:
# if runtime has GPU use GPU
if torch.cuda.is_available():
    device = torch.device("cuda:0")
else:
    device = torch.device("cpu")
print("Using device:", device)

In [ ]:
training_batch_size = 64
dataloader = data.DataLoader(dataset, batch_size=training_batch_size, shuffle=True, num_workers=2, drop_last=True)
dataloader_test = data.DataLoader(dataset_test, batch_size=training_batch_size, shuffle=True, num_workers=2, drop_last=True)

In [ ]:
def train(model, lr, criterion, trainloader, device, valloader = None, epochs=15):
    train_loss_over_epochs = []
    val_loss_over_epochs = []
    plt.ioff()
    fig = plt.figure()
    optimizer = optim.Adam(model.parameters(), lr=lr)
    for epoch in tqdm(range(epochs), total=epochs):
        running_loss = 0.0
        for i, data in enumerate(trainloader, 0):
            # Your code
            # -------------------------
            batch_x, batch_y = data
            if torch.cuda.is_available():
              batch_x = batch_x.cuda()
              batch_y = batch_y.cuda()
            optimizer.zero_grad()


            y_hat = model(batch_x)
            # print("batch_x:",np.shape(batch_x))
            # print("y_hat:",np.shape(y_hat))
            # print("y_hat:",y_hat[0])
            # print("batch_y:",np.shape(batch_y))
            y_hat = y_hat.squeeze()
            batch_y = batch_y.squeeze()
            batch_y = torch.argmax(batch_y, dim=1)
            loss = criterion(y_hat.float(), batch_y)


            loss.backward()
            optimizer.step()

            running_loss += loss.item() 
        running_loss /= len(trainloader)
            # -------------------------

        train_loss_over_epochs.append(running_loss)
        print('Epoch: {}, training loss: {:.3f}'.format(epoch + 1, running_loss))
        
    plt.subplot(2, 1, 1)
    plt.ylabel('Loss')
    plt.plot(np.arange(epochs), train_loss_over_epochs, color='red', label='train')
    if not valloader is None:
        plt.plot(np.arange(epochs), val_loss_over_epochs, color='blue', label='val')
    plt.title('Loss per Epoch')
    plt.xticks(np.arange(epochs, dtype=int))
    plt.grid(True)
    plt.legend()
    plt.show()
    return model



In [ ]:
class FSModel(nn.Module):
    def __init__(self, pretrained_resnet, num_layers_to_remove=0):
        super(FSModel, self).__init__()

        self.conv2d = nn.Conv2d(1, 3, 3, stride=1, padding=1)
        self.norm1 = nn.BatchNorm2d(3)
        self.maxpool = nn.MaxPool2d(3, stride=2, padding=1)

        self.pretrained_resnet = None
        if num_layers_to_remove != 0:
          self.pretrained_resnet = nn.Sequential(*list(pretrained_resnet.children())[:num_layers_to_remove])
        else:
          self.pretrained_resnet = pretrained_resnet
        
        for param in self.pretrained_resnet.parameters():
          param.requires_grad = False

        self.norm_res = nn.BatchNorm2d(64)

        self.conv2d2 = nn.Conv2d(3, 64, 3, stride=1, padding=1)
        self.norm2 = nn.BatchNorm2d(64)

        self.upsample = nn.Upsample(scale_factor=2, mode='bilinear', align_corners=True)

        self.conv2d3 = nn.Conv2d(128, 64, 3, stride=1, padding=1)
        self.norm3 = nn.BatchNorm2d(64)

        self.conv2d4 = nn.Conv2d(320,1, 3, stride=1, padding=1)
        self.norm4 = nn.BatchNorm2d(1)
        self.avgpool = nn.AvgPool2d(3, stride=1)

        self.fc = nn.Linear(180, SUB)

    def forward(self, x):
        N, C, T, P = np.shape(x)
        # print("0:", np.shape(x))
        x_x = self.maxpool(F.relu(self.norm1(self.conv2d(x[:,0,:,:].reshape((N,1,T,P)).float()))))
        # print("1:", np.shape(x_x))
        x_y = self.maxpool(F.relu(self.norm1(self.conv2d(x[:,1,:,:].reshape((N,1,T,P)).float()))))
        x_z = self.maxpool(F.relu(self.norm1(self.conv2d(x[:,2,:,:].reshape((N,1,T,P)).float()))))
        x_v = self.maxpool(F.relu(self.norm1(self.conv2d(x[:,3,:,:].reshape((N,1,T,P)).float()))))
        x_s = self.maxpool(F.relu(self.norm1(self.conv2d(x[:,4,:,:].reshape((N,1,T,P)).float()))))

        x_x_resnet = self.upsample(F.relu(self.norm_res(self.pretrained_resnet.forward(x_x))))
        # print("2:", np.shape(x_x_resnet))
        x_y_resnet = self.upsample(F.relu(self.norm_res(self.pretrained_resnet.forward(x_y))))
        x_z_resnet = self.upsample(F.relu(self.norm_res(self.pretrained_resnet.forward(x_z))))
        x_v_resnet = self.upsample(F.relu(self.norm_res(self.pretrained_resnet.forward(x_v))))
        x_s_resnet = self.upsample(F.relu(self.norm_res(self.pretrained_resnet.forward(x_s))))

        x_x = self.maxpool(F.relu(self.norm2(self.conv2d2(x_x))))
        # print("3:", np.shape(x_x))
        x_y = self.maxpool(F.relu(self.norm2(self.conv2d2(x_y))))
        x_z = self.maxpool(F.relu(self.norm2(self.conv2d2(x_z))))
        x_v = self.maxpool(F.relu(self.norm2(self.conv2d2(x_v))))
        x_s = self.maxpool(F.relu(self.norm2(self.conv2d2(x_s))))

        x_x = torch.cat([x_x, x_x_resnet], dim=1)
        # print("4:", np.shape(x_x))
        x_y = torch.cat([x_y, x_y_resnet], dim=1)
        x_z = torch.cat([x_z, x_z_resnet], dim=1)
        x_v = torch.cat([x_v, x_v_resnet], dim=1)
        x_s = torch.cat([x_s, x_s_resnet], dim=1)

        x_x = F.relu(self.norm3(self.conv2d3(x_x)))
        # print("5:", np.shape(x_x))
        x_y = F.relu(self.norm3(self.conv2d3(x_y)))
        x_z = F.relu(self.norm3(self.conv2d3(x_z)))
        x_v = F.relu(self.norm3(self.conv2d3(x_v)))
        x_s = F.relu(self.norm3(self.conv2d3(x_s)))

        x = torch.cat([x_x_resnet, x_y_resnet, x_z_resnet, x_v_resnet, x_s_resnet], dim=1)   
        # print("6:", np.shape(x))

        x= F.relu(self.norm4(self.conv2d4(x)))
        # print("7:", np.shape(x))

        x = self.avgpool(x)
        # print("8:", np.shape(x))
        x = x.reshape((N,180))
        

        x = self.fc(x)
        # print("6:", np.shape(x))

        # return nn.LogSoftmax(dim=1)(x)
        return x


In [ ]:
pretrained_resnet = models.resnet18(pretrained=True)

In [ ]:
FS_model = FSModel(pretrained_resnet, num_layers_to_remove=4).to(device)

In [ ]:
lr = 0.01
criterion = nn.CrossEntropyLoss()
FS_model = train(FS_model, lr, criterion, dataloader, device, epochs=40)

In [ ]:
# torch.save(FS_model, DIR + "/FS_model_40_40.p")
# FS_model = torch.load(DIR + "/FS_model_40_40.p")

In [ ]:
def validate_model(val_loader, model, device):
    preds = []
    gts = []
    with torch.no_grad():
        for data in val_loader:
            inputs, labels = data
            inputs = inputs.to(device)

            outputs = model(inputs).cpu().numpy()
            # return outputs, labels.numpy()
            preds += list(outputs)
            gts += list(labels.numpy())
            
    return preds, gts

preds, gts = validate_model(dataloader_test, FS_model, device)
len(np.argwhere(np.argmax(preds, axis=1) == np.argmax(gts, axis=1)))/len(gts)

In [ ]:
print(np.shape(preds))
print(np.shape(gts))

In [ ]:
class TSModel(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers):
        super(TSModel, self).__init__()

        def TDCNN(in_channels, out_channels):
          return nn.Sequential(
              nn.Conv1d(in_channels, out_channels, 3, padding=1),
              nn.BatchNorm1d(out_channels),
              nn.ReLU(inplace=True),
              nn.Conv1d(out_channels, out_channels, 3, padding=1),
              nn.BatchNorm1d(out_channels),
              nn.ReLU(inplace=True),
              nn.MaxPool1d(3, stride=1, padding=1)
          )

        self.tdcnn0 = TDCNN(5, 5)
        self.tdcnn1 = TDCNN(5, 5)
        self.tdcnn2 = TDCNN(5, 5)
        self.tdcnn3 = TDCNN(5, 5)

        self.tdcnn4 = TDCNN(5, 5)

        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True, bidirectional=True)
        # self.fc = nn.Linear(hidden_size*2, SUB)  
        self.fc = nn.Linear(320, SUB)  

    def forward(self, x):
        N, C, T, P = np.shape(x)

        # print("0: ",np.shape(x))
        assert T == 32, "Wrong TDCNN channel {}".format(C)

        x = torch.permute(x, (0,2,1,3)) # batch, T, C, P
        # print("1: ",np.shape(x))

        x_0 = self.tdcnn0(x[:,:8,:,:].reshape((N*8,C,P)).float()).reshape(N,8,-1)
        # print("2: ",np.shape(x_0))
        x_1 = self.tdcnn1(x[:,8:16,:,:].reshape((N*8,C,P)).float()).reshape(N,8,-1)
        x_2 = self.tdcnn2(x[:,16:24:,:,:].reshape((N*8,C,P)).float()).reshape(N,8,-1)
        x_3 = self.tdcnn3(x[:,24:,:,:].reshape((N*8,C,P)).float()).reshape(N,8,-1)

        x = torch.cat([x_0, x_1, x_2, x_3], dim=1)
        # print("3: ",np.shape(x))

        # x = self.tdcnn4(x)
        # print("2.2: ",np.shape(x))

        h0 = torch.zeros(self.num_layers*2, x.size(0), self.hidden_size).to(device)
        c0 = torch.zeros(self.num_layers*2, x.size(0), self.hidden_size).to(device)

        # x = x.reshape(N,T,-1)
        # print("4: ",np.shape(x))

        x, _ = self.lstm(x, (h0, c0)) 
        # print("4: ",np.shape(x))

        x = self.fc(x.reshape(N,-1))
        # print("5: ",np.shape(x))

        # assert False, "stop"
        
        return x


In [ ]:
input_size = 640
hidden_size = 5
num_layers = 2
TS_Model = TSModel(input_size, hidden_size, num_layers).to(device)

In [ ]:
lr = 0.01
criterion = nn.CrossEntropyLoss()
TS_Model = train(TS_Model, lr, criterion, dataloader, device, epochs=40)

In [ ]:
def validate_model(val_loader, model, device):
    preds = []
    gts = []
    with torch.no_grad():
        for data in val_loader:
            inputs, labels = data
            inputs = inputs.to(device)

            outputs = model(inputs).cpu().numpy()
            # return outputs, labels.numpy()
            preds += list(outputs)
            gts += list(labels.numpy())
            
    return preds, gts

preds, gts = validate_model(dataloader_test, TS_Model, device)
# preds, gts = validate_model(dataloader, TS_Model, device)

len(np.argwhere(np.argmax(preds, axis=1) == np.argmax(gts, axis=1)))/len(gts)